In [1]:
import sys
print(sys.executable)

c:\Users\00105010\Anaconda3\envs\loopflopy\python.exe


In [25]:
# Create new PEST folder
import os
from datetime import datetime

date_str = datetime.today().strftime('%Y-%m-%d')
folder_name = f"../pest"
os.makedirs(folder_name, exist_ok=True)

In [26]:
# Copy and paste PEST executable
import shutil
source = '../exe/pestpp-ies.exe'
destination = '../pest/pestpp-ies.exe'
shutil.copy(source, destination)

'../pest/pestpp-ies.exe'

In [27]:
# Move the BAT file into PEST folder
source = './Otorowiri_run_model.bat'
destination = '../pest/Otorowiri_run_model.bat'
shutil.copy(source, destination)

'../pest/Otorowiri_run_model.bat'

In [28]:
import os
import pyemu
import numpy as np
import pandas as pd

# Create template file
pars_df = pd.read_excel('../data/data_pest/pest_parameters_otorowiri.xlsx', sheet_name = 'pars')
pyemu.utils.helpers.simple_tpl_from_pars(pars_df.parnme.tolist(), 
                                         tplfilename='parameters.tpl', 
                                         out_dir='../pest')


In [29]:
# Create instruction file
obs_df = pd.read_excel('../data/data_pest/measured_groundwater.xlsx')
print(obs_df.columns.tolist())

obs_df['model_timestamp'] = obs_df['model_timestamp'].round(0).astype(int)

obs_df.columns = obs_df.columns.astype(str)
bore_cols = [c for c in obs_df.columns if c != 'model_timestamp']
print("Detected bore columns:", bore_cols)

obs_df = pd.melt(
    obs_df,
    id_vars=['model_timestamp'],
    value_vars=bore_cols,
    var_name='boreid',
    value_name='obsval')

obs_df = obs_df[pd.notna(obs_df['obsval'])]
obs_df = obs_df.reset_index(drop=True)
obs_df['obsnme'] = obs_df.apply(lambda row: f"{row['boreid']}_{row['model_timestamp']}", axis=1)
obs_df = obs_df.rename(columns={'value': 'obsval', 'variable': 'boreid'}) # Rename columns
obs_df['obgnme'] = 'head'
obs_df['weight'] = 1.0

pyemu.utils.helpers.simple_ins_from_obs(obsnames = obs_df.obsnme.tolist(), 
                                        insfilename='observations.ins', 
                                        out_dir='../pest')
obs_df

['model_timestamp', 61718118, 61718120, 61718121, 61718124, 61718126, 61719111, 61719112, 61819021, 61819023, 70118002, 70118003, 70118006, 70118009, 70118010, 70118014, 70118016, 70118017, 70119011, 70119012, 70119309]
Detected bore columns: ['61718118', '61718120', '61718121', '61718124', '61718126', '61719111', '61719112', '61819021', '61819023', '70118002', '70118003', '70118006', '70118009', '70118010', '70118014', '70118016', '70118017', '70119011', '70119012', '70119309']


,model_timestamp,boreid,obsval,obsnme,obgnme,weight
0,8840,61718118,217.945186,61718118_8840,head,1.0
1,9020,61718118,217.848936,61718118_9020,head,1.0
2,9264,61718118,217.892936,61718118_9264,head,1.0
3,9839,61718118,218.003936,61718118_9839,head,1.0
4,10202,61718118,218.123936,61718118_10202,head,1.0
...,...,...,...,...,...,...
1058,25704,70119309,228.296097,70119309_25704,head,1.0
1059,25976,70119309,228.516097,70119309_25976,head,1.0
1060,26550,70119309,229.061097,70119309_26550,head,1.0
1061,26914,70119309,229.711097,70119309_26914,head,1.0


In [30]:
# Define the basic components of the PEST control file
tpl_files = ['../pest/parameters.tpl']  # Template files
ins_files = ['../pest/observations.ins']  # Instruction files
par_files = ['parameters.par']  # Parameter files"
obs_files = ['observations.txt']  # Observation files

# Create a Pst object (PEST control file object)
pst = pyemu.Pst.from_io_files(
    tpl_files=tpl_files,
    in_files=par_files,
    ins_files=ins_files,
    out_files=obs_files,
    pst_path="."
)

# Add observation dataframe
pst.observation_data = obs_df
pst.observation_data

# Add parameter dataframe
pst.parameter_data = pars_df
pst.parameter_data

#print(pst.observation_data)
#print(pst.parameter_data)

c:\Users\00105010\Anaconda3\envs\loopflopy\Lib\site-packages\pyemu\pst\pst_utils.py:914: UserWarning: error processing instruction file ../pest/observations.ins, trying inschek: output file 'observations.txt' not found


,Type,ID,parnme,parlbnd,parval1,parubnd,partrans,parchglim,pargp,scale,offset,dercom
0,HK,1,k_kp,1.00000,8.0000,100.000,log,factor,HK,1,0,1
1,VK,2,vk_kp,0.01000,0.1000,10.000,log,factor,VK,1,0,1
2,SY,3,sy,0.05000,0.1000,0.500,none,factor,SY,1,0,1
3,SS,4,ss,0.00001,0.0001,0.001,none,factor,SS,1,0,1
4,RCH,5,rch_woody_coeff,0.00500,0.0100,0.050,none,factor,RCH,1,0,1
5,RCH,6,rch_nonwoody_coeff,0.01000,0.0600,0.100,none,factor,RCH,1,0,1
6,EVT,7,evt_woody_multiplier,0.10000,0.6000,1.000,none,factor,EVT,1,0,1
7,EVT,8,evt_nonwoody_multiplier,0.80000,1.3000,1.800,none,factor,EVT,1,0,1
8,EVT,9,evt_woody_depth,3.00000,5.0000,10.000,none,factor,EVT,1,0,1
9,EVT,10,evt_nonwoody_depth,0.10000,2.0000,5.000,none,factor,EVT,1,0,1


In [31]:
#pst.control_data.formatted_values
pst.model_command = ["Otorowiri_run_model.bat"]  # Model command to run
pst.control_data.noptmax = 5 # Max iterations (-2 = Single realisation)
#pst.control_data.relparmax = 1.5  # Maximum relative parameter change
pst.control_data.phiredstp = 0.01  # will stop at 1% improvement
pst.control_data.nphistp = 2  # will tolerate n slow-improvements
pst.control_data.nphinored = 2  # will tolerate n non-improvements

In [32]:
# ENSEMBLE
pst.pestpp_options['ies_num_reals'] = 3          # Number of ensembles (ne)
#pst.pestpp_options['ies_n_iter_base'] = 5       # Base iterations (minimum)
#pst.pestpp_options['ies_n_iter_super'] = 20     # Super iterations (maximum)
#pst.pestpp_options['ies_n_iter_mean'] = 10      # Mean target iterations

In [33]:
# LAMBDA
pst.pestpp_options["lambdas"] = [0.1, 1, 10, 100]     # Base lambda values to test
#pst.pestpp_options['ies_lambda_scale_fac'] = 1.0      # Global scaling factor
#pst.pestpp_options['ies_lambda_mults'] = [0.1, 1, 10] # Additional multipliers

# LAMBDA SCALE FACTOR:
# 0.1 - 0.5:  Aggressive (larger parameter updates)
# 1.0:        Default (no scaling)
# 2.0 - 10.0: Conservative (smaller parameter updates)

In [34]:
pst.pestpp_options['ies_verbose_level'] = 3  # 0=low, 1=medium, 2=high, 3=very high
#pst.pestpp_options['forecasts'] = []  # 'drawdown'
#pst.pestpp_options['da_use_simulated_states'] = True # whether to use the simulated states at the end of each cycle as the initial states for the next cycle.
#pst.pestpp_options.pop("forecasts",None)
#pst.pestpp_options['upgrade_augment'] = True  # Ill-conditioned, unstable
pst.pestpp_options.pop("upgrade_augment",None) # If well conditioned, basic algorithm
#pst.pestpp_options['ies_parameter_ensemble'] = 'dummy_ies_par_ensemble.csv'

print(pst.nnz_obs_groups)
print(pst.nnz_obs, pst.npar_adj)
print(pst.adj_par_groups)
pst.pestpp_options

['head']
1063 11
['HK', 'VK', 'SY', 'SS', 'RCH', 'EVT', 'DRN']


{'ies_num_reals': 3, 'lambdas': [0.1, 1, 10, 100], 'ies_verbose_level': 3}

In [ ]:
pst.write('../pest/Otorowiri.pst', version=2) #writing the pest control file

noptmax:5, npar_adj:11, nnz_obs:1063
